In [347]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn import naive_bayes
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
import pandas as pd
from nltk.corpus import wordnet
from nltk.tag import pos_tag
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import svm
import math
import numpy as np

In [56]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\utkarsh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [91]:
df=pd.read_csv("0000000000002747_training_twitter_x_y_train.csv")

In [92]:
y=df.airline_sentiment


In [93]:
#negative reason gold is useless, airline sentiment gold is a perfect prediction feature

In [94]:
df.drop(columns=["airline_sentiment_gold","negativereason_gold","airline_sentiment","tweet_id","name"],axis=1,inplace=True)

In [95]:
def label(x):
    if x=="negative":
        return 1
    elif x=="neutral":
        return 2
    else:
        return 3


def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB    
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


In [96]:
y=y.apply(label)

In [97]:
x=list(df["text"])
lem=WordNetLemmatizer()

In [411]:
new_x=[]
for i in x:
    temp=pos_tag(word_tokenize(i))
    fg=[]
    for j in temp:
        fg.append(lem.lemmatize(j[0],pos=get_wordnet_pos(j[1])))
    new_x.append(' '.join(fg))

In [99]:
stop=set(stopwords.words("english"))

In [100]:
punc=set(string.punctuation)

In [101]:
stop.update(punc)

In [412]:
vec=CountVectorizer(max_features=5000,stop_words=stop,ngram_range=(1,2),max_df=0.8,min_df=0.001)

In [413]:
new_x1=vec.fit_transform(new_x)

In [414]:

model=RandomForestClassifier(n_estimators=2000, n_jobs=-1)


In [415]:
model.fit(new_x1,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=2000,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [302]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           1       0.92      0.79      0.85      2073
           2       0.57      0.78      0.66       675
           3       0.71      0.76      0.73       546

    accuracy                           0.78      3294
   macro avg       0.73      0.77      0.75      3294
weighted avg       0.81      0.78      0.79      3294



In [303]:
print(confusion_matrix(y_test,pred))

[[1635  318  120]
 [  97  525   53]
 [  49   83  414]]


In [416]:
test=pd.read_csv("0000000000002747_test_twitter_x_test.csv")

In [417]:
test=test[["airline_sentiment_gold","text"]]

In [320]:
def sentlabel(x):
    if x=="negative":
        return 1
    elif x=="neutral":
        return 2
    elif x=="positive":
        return 3
    return 0

In [380]:
impose=test['airline_sentiment_gold'].copy()

In [418]:
test.drop('airline_sentiment_gold',axis=1,inplace=True)

In [419]:
new_x=[]
for i in test["text"]:
    temp=pos_tag(word_tokenize(i))
    fg=[]
    for j in temp:
        fg.append(lem.lemmatize(j[0],pos=get_wordnet_pos(j[1])))
    new_x.append(' '.join(fg))

In [420]:
xt=vec.transform(new_x)

In [421]:
ans=model.predict(xt)

In [422]:
d={1:'negative',2:'neutral',3:'positive'}
pred=[d[i] for i in ans]

In [423]:
pred=pd.DataFrame(pred)


In [424]:
pred.to_csv("pred.csv",header=False,
    index=False)

1